In [6]:
import random
import time
from datetime import datetime
from difflib import SequenceMatcher
import cv2
import easyocr
from private.secrets import KEY, LOCAL
import imutils
import numpy as np
import pandas as pd
import requests
from matplotlib import pyplot as plt

In [14]:
!"./initial.bat"

# Load the model
net = cv2.dnn.readNet("./Data/plates-ssd.xml", "./Data/plates-ssd.bin")

# Specify target device
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Read an image


(base) d:\A\Deploy>C:\"Program Files (x86)"\Intel\openvino_2021.4.752\bin\setupvars.bat
Python 3.9.7
[setupvars.bat] OpenVINO environment initialized


error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\dnn.cpp:4319: error: (-2:Unspecified error) Build OpenCV with Inference Engine to enable loading models from Model Optimizer. in function 'cv::dnn::dnn4_v20211220::Net::readFromModelOptimizer'


In [2]:
def sms(number: list, link: str):
    url = "https://www.fast2sms.com/dev/bulkV2"

    payload = f"sender_id=FSTSMS&message={link}&language=english&route=q&numbers={','.join(number)}"

    headers = {
        "authorization": KEY,
        "Content-Type": "application/x-www-form-urlencoded",
        "Cache-Control": "no-cache",
    }

    response = requests.request("POST", url, data=payload, headers=headers)

    return response.text

In [3]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [4]:
video = cv2.VideoCapture("Data/Deploy01.mp4")

database = pd.read_csv("data.csv")
results = pd.read_csv("results.csv")
history = ""
nullCheck = 0
firstFrame = None
motion = False

plates = pd.DataFrame(columns=["Time", "Plates"])

NameError: name 'cv2' is not defined

In [5]:
# sms(['9006818029', '7772966141', '9123415629'], 'testing multiple numbers')

In [13]:
k = 0
while k != 20:  # TODO :: CHANGE IN PROD
    if nullCheck >= 50:
        break

    ret, img = video.read()

    cv2.imshow("og", img)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    if firstFrame is None:
        firstFrame = gray
        continue

    delta = cv2.absdiff(firstFrame, gray)
    thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)

    for c in cnts:
        # if the contour is too small, ignore it
        if cv2.contourArea(c) < 500:
            motion = False
            continue
        else:
            motion = True

    if motion:
        bfilter = cv2.bilateralFilter(gray, 11, 17, 17)
        edged = cv2.Canny(bfilter, 30, 200)

        keypoints = cv2.findContours(
            edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE
        )
        contours = imutils.grab_contours(keypoints)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

        location = None
        for contour in contours:
            approx = cv2.approxPolyDP(contour, 10, True)
            if len(approx) == 4:
                location = approx
                break

        mask = np.zeros(gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [location], 0, 255, -1)
        new_image = cv2.bitwise_and(img, img, mask=mask)
        new = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)

        (x, y) = np.where(mask == 255)
        (x1, y1) = (np.min(x), np.min(y))
        (x2, y2) = (np.max(x), np.max(y))
        cropped_image = gray[x1 : x2 + 1, y1 : y2 + 1]

        reader = easyocr.Reader(["en"])
        result = reader.readtext(cropped_image)

        try:
            plate = result[0][1]
            print(plate)
            if plate in plate in database.Plate.to_string():
                conInfo = database.iloc[
                    np.where(database["Plate"] == plate)
                ].Contact.to_string(index=False)

            if history != plate:
                sms([conInfo], "testing")  # TODO :: Change in prod

            nullCheck = 0
            history = result[0][1]

        except Exception as e:
            print("Not Found !!!")
            nullCheck += 1
            print(e)

    time.sleep(10)  # TODO :: Change in production
    k += 1

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


True
Hui
Not Found !!!
list index out of range
False
False


KeyboardInterrupt: 